###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

In [1]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import html
import numpy as np

## Caso 1: Scraping de un catálogo: Labirratorium

In [3]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [5]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [4]:
r = requests.get(url)
soup = bs("", 'lxml')

In [7]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_="")

12


In [10]:
print(cervezas_grid[0])

<div class="product-image">
<!-- @file modules\appagebuilder\views\templates\front\products\file_tpl -->
<ul class="product-flags">
<li class="product-flag on-sale">¡En oferta!</li>
</ul>
<a class="thumbnail product-thumbnail" href="https://www.labirratorium.com/es/cervezas-por-estilo/617-espiga-mosaic.html">
<img alt="Espiga Blonde Sin Gluten" class="img-fluid" data-full-size-image-url="https://www.labirratorium.com/980-large_default/espiga-mosaic.jpg" src="https://www.labirratorium.com/980-home_default/espiga-mosaic.jpg"/>
<span class="product-additional" data-idproduct="617"></span>
</a>
</div>


In [13]:
# necesitamos acceder a cada una de las cervezas del grid:
lista_URLs = []
for cerveza in cervezas_grid:
    URL_cerveza = cerveza.find('')['']
    lista_URLs.append(URL_cerveza)

lista_URLs

['https://www.labirratorium.com/es/cervezas-por-estilo/617-espiga-mosaic.html',
 'https://www.labirratorium.com/es/inicio/355-kwak-33cl.html',
 'https://www.labirratorium.com/es/inicio/12180-hacker-pschorr-oktoberfest.html',
 'https://www.labirratorium.com/es/inicio/3852-grimbergen-optimo-bruno.html',
 'https://www.labirratorium.com/es/inicio/1343-cantillon-kriek-75cl.html',
 'https://www.labirratorium.com/es/inicio/16042-goeller-kellerbier.html',
 'https://www.labirratorium.com/es/inicio/10345-beavertown-neck-oil.html',
 'https://www.labirratorium.com/es/inicio/6716-lindemans-oude-gueuze-cuvee-rene-blend-2017.html',
 'https://www.labirratorium.com/es/inicio/16650-faust-krausen-naturtrub.html',
 'https://www.labirratorium.com/es/inicio/8446-de-cam-framboise-lambiek.html',
 'https://www.labirratorium.com/es/inicio/2073-belhaven-90.html',
 'https://www.labirratorium.com/es/inicio/628-the-lost-abbey-serpents-stout.html']

In [15]:
# Hacemos un nuevo request para la primera cerveza: 
r = requests.get(lista_URLs[0])

# Creamos una sopa específica con la info de cada cerveza
soup_cerveza = bs("", "lxml")

In [17]:
# Nombre
name = soup_cerveza.find(class_="").
print(name)

Espiga Hop Collection Series Mosaic


In [21]:
# Precio
price = soup_cerveza.find(class_="current-price").find
print(price)

3.2


In [26]:
# Descripcion corta
try: 
    descr_short = soup_cerveza.find(class_="description-short")
except:
    descr_short = None

print(descr_short)

Pale Ale


In [30]:
# Descripción larga
descr_full_list = []
descr_full_p = soup_cerveza.find(class_="product-description")
for sentence in descr_full_p:
    descr_full_list.append(sentence.text)

descr_full = "\n".join(descr_full_list)

print(descr_full)

Espiga Hop Collection Mosaic Edition
Perteneciente a la Hop Collection Series, la 1, monovarietal de Mosaic
Cerveza rubia de cuerpo ligero medio, con notas afrutadas y herbales. 


In [31]:
# Imagen
image = soup_cerveza.find(class_="js-qv-product-cover img-fluid")
print(image)

https://www.labirratorium.com/980-large_default/espiga-mosaic.jpg


In [32]:
 # Brand

try:
    brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')
except:
    brand = None
print(brand)

Espiga


In [35]:
# Código de barras
try:
    barcode = soup_cerveza.find(class_='product-reference')
except:
    barcode = None
print(barcode)

617


In [36]:
# Features
features_dicc = {}

features = soup_cerveza.find(class_="data-sheet")

try:
    features_dt = features.find_all("dt")
    features_dd = features.find_all("dd")
    for feature, value in zip(features_dt, features_dd):
        print(feature.text, value.text)
        features_dicc[feature.text] = value.text
except:
    features_dicc = {}

Estilo APA (AMERICAN PALE ALE)
Origen Barcelona
% Alc. MEDIO (4-6%)
% Alc. Exacto 5.5
Volumen (cl) 33 Cl
Tipo Fermentación Ale (Alta Fermentación)
Maltas Cebada
Lúpulos Mosaic
IBU 25-50 Amargor medio
IBU Exacto 25
Color Amarillo pálido
Envase Botella
OLD-REF 8425402460207


In [46]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD
counter = 1
id_cerv = "lbt_" + str(counter)

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [47]:
# Agregamos a una lista
lista_cervezas = []

lista_cervezas.append([
    ("id", id_cerv),
    ("name", name),
    ('price', price),
    ('descr_short', descr_short),
    ('descr_full', descr_full),
    ('image', image),
    ('brand', brand),
    ('barcode', barcode),
    ('features', features_dicc)
])

lista_cervezas

[[('id', 'lbt_1'),
  ('name', 'Espiga Hop Collection Series Mosaic'),
  ('price', '3.2'),
  ('descr_short', 'Pale Ale'),
  ('descr_full',
   'Espiga Hop Collection Mosaic Edition\nPerteneciente a la Hop Collection Series, la 1, monovarietal de Mosaic\nCerveza rubia de cuerpo ligero medio,\xa0con notas afrutadas y herbales.\xa0'),
  ('image',
   'https://www.labirratorium.com/980-large_default/espiga-mosaic.jpg'),
  ('brand', 'Espiga'),
  ('barcode', '617'),
  ('features',
   {'Estilo': 'APA (AMERICAN PALE ALE)',
    'Origen': 'Barcelona',
    '% Alc.': 'MEDIO (4-6%)',
    '% Alc. Exacto': '5.5',
    'Volumen (cl)': '33 Cl',
    'Tipo Fermentación': 'Ale (Alta Fermentación)',
    'Maltas': 'Cebada',
    'Lúpulos': 'Mosaic',
    'IBU': '25-50 Amargor medio',
    'IBU Exacto': '25',
    'Color': 'Amarillo pálido',
    'Envase': 'Botella',
    'OLD-REF': '8425402460207'})]]

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

In [50]:
pages = np.arange(1, 10)
count = 1
lista_cervezas = []

for page in pages:
    
    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page=' + str(page)
    r = requests.get(URL)
    soup = bs(r.text, 'lxml')
    cervezas_grid = soup.find_all(class_='product-image')
    
    count_beer = 1 # para el print de seguimiento de descarga
    
    for cerveza in cervezas_grid:
        # Print de seguimiento de descarga:
        print('Cerveza {} de {}, pag {}/{}'.format(
            count_beer, len(cervezas_grid), page, len(pages)))

        URL_cerveza = cerveza.find('a')['href']
        r = requests.get(URL_cerveza)
        soup_cerveza = bs(r.text, 'lxml')
        
        id_cerv = 'lbt_' + str(count)
        
        name = soup_cerveza.find(class_='h1 product-detail-name').text   

        price = soup_cerveza.find(class_='current-price').find('span')['content']

        try:
            descr_short = soup_cerveza.find(class_='description-short').find('p').text
        except:
            descr_short = None

        descr_full_list = []
        descr_full_p = soup_cerveza.find(class_='product-description').find_all('p')
        for sentence in descr_full_p:
            descr_full_list.append(sentence.text)
            
        descr_full = "\n".join(descr_full_list)
        
        image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']

        try:
            brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
        except:
            brand = None

        try:
            barcode = soup_cerveza.find(class_='product-reference').find('span').text
        except:
            barcode = None

        features_dicc = {}
        features = soup_cerveza.find(class_ = 'data-sheet')
        try:
            features_dt = features.find_all('dt')
            features_dd = features.find_all('dd')
            for feature, value in zip(features_dt, features_dd):
                #print(feature.text, value.text)
                features_dicc[feature.text] = value.text
        except:
            features_dicc = {}

        lista_cervezas.append([
            ('id', id_cerv),
            ('name', name), 
            ('price', price),
            ('descr_short', descr_short),
            ('descr_full', descr_full),
            ('image', image),
            ('brand', brand),
            ('barcode', barcode),
            ('features', features_dicc)
        ])        
        
        # Pasamos al siguiente id
        count += 1
        count_beer += 1

Cerveza 1 de 12, pag 1/80
Cerveza 2 de 12, pag 1/80
Cerveza 3 de 12, pag 1/80
Cerveza 4 de 12, pag 1/80
Cerveza 5 de 12, pag 1/80
Cerveza 6 de 12, pag 1/80
Cerveza 7 de 12, pag 1/80
Cerveza 8 de 12, pag 1/80
Cerveza 9 de 12, pag 1/80
Cerveza 10 de 12, pag 1/80
Cerveza 11 de 12, pag 1/80
Cerveza 12 de 12, pag 1/80
Cerveza 1 de 12, pag 2/80
Cerveza 2 de 12, pag 2/80


KeyboardInterrupt: 

Construir un DataFrame